# 다음 영화 크롤링

In [61]:
from time import sleep
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import csv
import re
import pandas as pd

In [4]:
'''
    <주의사항>
    - 2019년에는 평론가 평점이 없는 경우가 많음.
    - 2018년 '데드풀2'는 상세정보 열람이 불가능
    - 결측치는 'NA'로 처리
'''

In [172]:
def daum_movie_crawler(year):
    CHROME_DIR = '/Users/youngerous/young/bin/chromedriver'
    browser = webdriver.Chrome(CHROME_DIR)
    browser.implicitly_wait(5)
    browser.get('https://movie.daum.net/boxoffice/yearly?year='+str(year))
    sleep(1)
    
    # 변수 설정
    movie_name = []
    movie_rating_netizen = []
    movie_rating_columnist = []
    movie_total_audience = []
    num_movies = len(browser.find_elements_by_class_name('name_movie'))
    
    for idx in range(num_movies):
        movie_list = browser.find_elements_by_class_name('name_movie')
        movie = movie_list[idx]
        
        # 영화 제목 추가 후 상세 페이지로 이동
        print('[', idx+1, movie.text,']') # 크롤링 위치 확인을 위해 삽입하였습니다.
        movie_name.append(movie.text)
        movie.click() 
        sleep(1)

        try:
            # 누적 관객 수
            total = browser.find_element_by_id('totalAudience')
            total_num = extract_number_from_string(total.text)
            movie_total_audience.append(total_num)
        
            try:
                # 네티즌 평점 / 평론가 평점
                ratings = browser.find_elements_by_class_name('score_rating')
                movie_rating_columnist.append(float(ratings[1].text)) # 전문가 평점 누락 시 Exception을 발생시키기 위해 1 index를 먼저 append
                movie_rating_netizen.append(float(ratings[0].text))
            except:
                print('* 전문가 평점 누락')
                ratings = browser.find_element_by_class_name('score_rating')
                movie_rating_netizen.append(float(ratings.text))
                movie_rating_columnist.append('NA')
        
        except:
            print('** 정보 제공 불가 항목')
            movie_rating_netizen.append('NA')
            movie_rating_columnist.append('NA')
            movie_total_audience.append('NA')
            
        # 이전 페이지로 돌아간다.
        browser.back()
        sleep(1)
    
    # 세션을 종료한다. 
    browser.quit()
    return make_dataframe(year, movie_name, movie_rating_netizen, movie_rating_columnist, movie_total_audience)
    

In [173]:
# 문자 내에서 숫자를 추출하여 합치는 함수
def extract_number_from_string(text):
    numbers = re.findall('\d+', text)
    return int("".join(numbers))

# 크롤링 결과 리스트를 DataFrame으로 만드는 함수
def make_dataframe(year, name, netizen, columnist, audience):
    result = pd.DataFrame()
    length = len(name)
    
    result['year']             = [year]*length
    result['title']            = name
    result['rating_netizen']   = netizen
    result['rating_columnist'] = columnist
    result['audience']         = audience
    
    return result

In [174]:
movie_2018 = daum_movie_crawler(2018)
movie_2019 = daum_movie_crawler(2019)

[ 1 신과함께-인과 연 ]
[ 2 어벤져스: 인피니티 워 ]
[ 3 보헤미안 랩소디 ]
[ 4 미션 임파서블: 폴아웃 ]
[ 5 신과함께-죄와 벌 ]
[ 6 쥬라기 월드: 폴른 킹덤 ]
[ 7 앤트맨과 와스프 ]
[ 8 안시성 ]
[ 9 블랙 팬서 ]
[ 10 완벽한 타인 ]
[ 11 1987 ]
[ 12 독전 ]
[ 13 공작 ]
[ 14 베놈 ]
[ 15 암수살인 ]
[ 16 데드풀 2 ]
** 정보 제공 불가 항목
[ 17 국가부도의 날 ]
[ 18 코코 ]
[ 19 아쿠아맨 ]
[ 20 그것만이 내 세상 ]
[ 21 마녀 ]
[ 22 탐정: 리턴즈 ]
[ 23 인크레더블 2 ]
[ 24 서치 ]
[ 25 너의 결혼식 ]
[ 26 곤지암 ]
[ 27 지금 만나러 갑니다 ]
[ 28 목격자 ]
[ 29 조선명탐정: 흡혈괴마의 비밀 ]
[ 30 신비한 동물들과 그린델왈드의 범죄 ]
[ 31 메이즈 러너: 데스 큐어 ]
[ 32 맘마미아!2 ]
[ 33 레디 플레이어 원 ]
[ 34 명당 ]
[ 35 협상 ]
[ 36 마약왕 ]
[ 37 쥬만지: 새로운 세계 ]
[ 38 창궐 ]
[ 39 성난황소 ]
[ 40 도어락 ]
[ 41 리틀 포레스트 ]
[ 42 골든슬럼버 ]
[ 43 램페이지 ]
[ 44 궁합 ]
[ 45 오션스8 ]
[ 46 사라진 밤 ]
[ 47 스윙키즈 ]
[ 48 바람 바람 바람 ]
[ 49 PMC: 더 벙커 ]
[ 50 범블비 ]
[ 1 극한직업 ]
* 전문가 평점 누락
[ 2 어벤져스: 엔드게임 ]
* 전문가 평점 누락
[ 3 알라딘 ]
* 전문가 평점 누락
[ 4 기생충 ]
* 전문가 평점 누락
[ 5 스파이더맨: 파 프롬 홈 ]
* 전문가 평점 누락
[ 6 엑시트 ]
* 전문가 평점 누락
[ 7 캡틴 마블 ]
* 전문가 평점 누락
[ 8 라이온 킹 ]
* 전문가 평점 누락
[ 9 봉오동 전투 ]
* 전문가 평점 누락
[ 10 토이 스토리 4 ]
* 전문가 평점 누락
[ 11 돈 ]
* 전문가 평점 누락
[ 12 악인전 ]
* 전문가 

In [175]:
movie_2018

,year,title,rating_netizen,rating_columnist,audience
0,2018,신과함께-인과 연,6.8,6.4,12276115
1,2018,어벤져스: 인피니티 워,7.6,7.3,11212710
2,2018,보헤미안 랩소디,8.9,6.3,9948386
3,2018,미션 임파서블: 폴아웃,8.1,7.7,6584919
4,2018,신과함께-죄와 벌,7,6.1,14411675
5,2018,쥬라기 월드: 폴른 킹덤,6.8,6.2,5661128
6,2018,앤트맨과 와스프,7.4,6.3,5448134
7,2018,안시성,7.6,6.4,5441020
8,2018,블랙 팬서,6.1,6.9,5399327
9,2018,완벽한 타인,7.5,6.6,5294154


In [176]:
movie_2019

,year,title,rating_netizen,rating_columnist,audience
0,2019,극한직업,7.4,NA,16264806
1,2019,어벤져스: 엔드게임,7.8,NA,13934545
2,2019,알라딘,8.4,NA,12508150
3,2019,기생충,7.8,NA,10080207
4,2019,스파이더맨: 파 프롬 홈,6.6,NA,8020052
5,2019,엑시트,7.9,NA,7753662
6,2019,캡틴 마블,6.3,NA,5801070
7,2019,라이온 킹,7.4,NA,4736167
8,2019,봉오동 전투,8.8,NA,4125290
9,2019,토이 스토리 4,8.7,NA,3395872


In [177]:
movie = pd.concat([movie_2018, movie_2019])
movie

,year,title,rating_netizen,rating_columnist,audience
0,2018,신과함께-인과 연,6.8,6.4,12276115
1,2018,어벤져스: 인피니티 워,7.6,7.3,11212710
2,2018,보헤미안 랩소디,8.9,6.3,9948386
3,2018,미션 임파서블: 폴아웃,8.1,7.7,6584919
4,2018,신과함께-죄와 벌,7,6.1,14411675
5,2018,쥬라기 월드: 폴른 킹덤,6.8,6.2,5661128
6,2018,앤트맨과 와스프,7.4,6.3,5448134
7,2018,안시성,7.6,6.4,5441020
8,2018,블랙 팬서,6.1,6.9,5399327
9,2018,완벽한 타인,7.5,6.6,5294154


In [178]:
movie.to_csv('movie.csv', header=['year', 'title', 'rating_netizen', 'rating_columnist', 'audience'], index=False)